In [1]:
import os
os.chdir('../')

In [2]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *

Loading imports


In [3]:
torch.cuda.set_device(3)

In [4]:
Meta.df_comb.head()

,Modality,PatientID,StudyInstanceUID,SeriesInstanceUID,StudyID,ImagePositionPatient,ImageOrientationPatient,SamplesPerPixel,PhotometricInterpretation,Rows,...,MultiWindowCenter,WindowCenter1,MultiWindowWidth,WindowWidth1,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
SOPInstanceUID,,,,,,,,,,,,,,,,,,,,,
ID_76d55d9d0,CT,ID_b9797064,ID_00b9e1961f,ID_0000298a7d,,-125.0,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_96d282ea9,CT,ID_b9797064,ID_00b9e1961f,ID_0000298a7d,,-125.0,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_7d8a7c29d,CT,ID_b9797064,ID_00b9e1961f,ID_0000298a7d,,-125.0,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_4d4401491,CT,ID_b9797064,ID_00b9e1961f,ID_0000298a7d,,-125.0,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_8f5ded0b7,CT,ID_b9797064,ID_00b9e1961f,ID_0000298a7d,,-125.0,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0


## Training

In [5]:
name = 'baseline_full_stg1_album'

In [6]:
dbch = get_album_data(512, 128, splits=Meta.splits_stg1)
learn = get_learner(dbch, xresnet18, pretrained=True)

## Training

In [7]:
do_fit(learn, 12, 4e-2)
learn.save(f'runs/{name}-1')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.121825,0.322755,0.887052,0.679458,08:53
1,0.110340,0.120292,0.965347,0.927825,08:48
2,0.105973,0.140129,0.961175,0.904946,08:23
3,0.100634,0.124001,0.965742,0.925138,07:24
4,0.098250,0.100677,0.969928,0.939207,09:44
5,0.093742,0.110747,0.968751,0.934674,07:56
6,0.089869,0.103589,0.968786,0.935298,07:44
7,0.087077,0.095993,0.971051,0.940760,08:46
8,0.080710,0.096636,0.972432,0.942135,08:16
9,0.075985,0.084212,0.974353,0.948857,08:18


In [8]:
learn.dbunch = get_album_data(512, 256, splits=Meta.splits_stg1)
do_fit(learn, 12, 4e-3, freeze=False)
learn.save(f'runs/{name}-2')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.082955,0.088718,0.972837,0.946884,13:29
1,0.077596,0.088343,0.973034,0.945624,13:25
2,0.075475,0.084964,0.974649,0.949952,13:28
3,0.072566,0.088011,0.973750,0.946540,13:30
4,0.071137,0.081148,0.975042,0.951633,13:22
5,0.069857,0.078381,0.975993,0.952728,13:22
6,0.067326,0.079218,0.975907,0.953148,13:18
7,0.063690,0.080946,0.975890,0.952626,13:20
8,0.060109,0.081026,0.976638,0.954637,13:30
9,0.057265,0.080089,0.976285,0.953912,13:24


In [9]:
learn.dbunch = get_album_data(256, 384, splits=Meta.splits_stg1, img_dir=path_jpg)
do_fit(learn, 4, 4e-4, freeze=False)
learn.save(f'runs/{name}-3')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.066584,0.081532,0.975887,0.952740,29:56
1,0.062915,0.078979,0.975916,0.952906,29:40
2,0.057435,0.077244,0.976688,0.954981,29:46
3,0.056702,0.078346,0.976659,0.954370,29:41


## Submission

In [7]:
learn.load(f'runs/{name}-3')

In [8]:
# tst_fns = df_tst.index.values

In [9]:
sub_fn = f'subm/{name}-3'

In [10]:
learn.dbunch = get_test_data(Meta.df_tst, bs=512, sz=384)

In [11]:
# tst_bs = 256
# tst_sz = 384
# tst_splits = [L.range(tst_fns), L.range(tst_fns)]
# tst_dbch = get_data_gen(tst_fns, bs=tst_bs, img_tfm=get_pil_fn(path/'tst_jpg'), sz=tst_sz, splits=tst_splits, test=True)
# learn.dbunch = tst_dbch

In [12]:
preds,targs = learn.get_preds()

In [13]:
pred_csv = submission(Meta.df_tst, preds, fn=sub_fn)

In [14]:
FileLink(f'{sub_fn}.csv')

/home/ashaw/kaggle/rsna_retro/subm/baseline_full_stg1_album-3.csv

In [16]:
api.competition_submit(f'{sub_fn}.csv', 'Training on Stage 1 (384), Valid on Stage 2. - 0.087925', 'rsna-intracranial-hemorrhage-detection')

100%|██████████| 26.0M/26.0M [00:02<00:00, 9.90MB/s]


Successfully submitted to RSNA Intracranial Hemorrhage Detection

In [21]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 14266098,
 'totalBytes': 27277209,
 'date': '2020-01-22T18:41:10.043Z',
 'description': 'Training on Stage 1 (384), Valid on Stage 2. - 0.087925',
 'errorDescription': None,
 'fileName': 'baseline_full_stg1_album-3.csv',
 'publicScore': '0.52899',
 'privateScore': '0.06902',
 'status': 'complete',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/14266098/14266098.raw'}